In [1]:
from minio import Minio

client = Minio(
    "localhost:9000",
    access_key="dev_user_01",
    secret_key="devpassword01",
    secure=False
)

bucket_name = "test-bucket"

found = client.bucket_exists(bucket_name)
if not found:
    client.make_bucket(bucket_name)

In [2]:
destination_file = 'test.csv'
source_file = 'test.csv'

client.fput_object(bucket_name, destination_file, source_file,)

In [3]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName('test') \
    .getOrCreate()
    
#.config("spark.executor.memory", "2g") \
#.config("spark.driver.memory", "2g") \

your 131072x1 screen size is bogus. expect trouble
24/03/31 17:35:09 WARN Utils: Your hostname, DESKTOP-IEOE9QF resolves to a loopback address: 127.0.1.1; using 172.23.154.177 instead (on interface eth0)
24/03/31 17:35:09 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/03/31 17:35:11 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [4]:
spark.sparkContext.getConf().getAll()

[('spark.executor.memory', '2g'),
 ('spark.driver.port', '42683'),
 ('spark.app.name', 'test'),
 ('spark.executor.id', 'driver'),
 ('spark.driver.extraJavaOptions',
  '-Djava.net.preferIPv6Addresses=false -XX:+IgnoreUnrecognizedVMOptions --add-opens=java.base/java.lang=ALL-UNNAMED --add-opens=java.base/java.lang.invoke=ALL-UNNAMED --add-opens=java.base/java.lang.reflect=ALL-UNNAMED --add-opens=java.base/java.io=ALL-UNNAMED --add-opens=java.base/java.net=ALL-UNNAMED --add-opens=java.base/java.nio=ALL-UNNAMED --add-opens=java.base/java.util=ALL-UNNAMED --add-opens=java.base/java.util.concurrent=ALL-UNNAMED --add-opens=java.base/java.util.concurrent.atomic=ALL-UNNAMED --add-opens=java.base/sun.nio.ch=ALL-UNNAMED --add-opens=java.base/sun.nio.cs=ALL-UNNAMED --add-opens=java.base/sun.security.action=ALL-UNNAMED --add-opens=java.base/sun.util.calendar=ALL-UNNAMED --add-opens=java.security.jgss/sun.security.krb5=ALL-UNNAMED -Djdk.reflect.useDirectMethodHandle=false'),
 ('spark.driver.host', '

In [4]:
coffee_csv = 'coffee_sales.csv'
coffee_df = spark.read.format('csv').option('header', 'true').load(coffee_csv)
coffee_df.count()

149116

In [10]:
coffee_df.show()

+--------------+----------------+----------------+--------+--------------+----------+---------------+----------+----------+----------------+-----------------+--------------+-----+----------+---------+----+-----+-----------+
|transaction_id|transaction_date|transaction_time|store_id|store_location|product_id|transaction_qty|unit_price|Total_Bill|product_category|     product_type|product_detail| Size|Month Name| Day Name|Hour|Month|Day of Week|
+--------------+----------------+----------------+--------+--------------+----------+---------------+----------+----------+----------------+-----------------+--------------+-----+----------+---------+----+-----+-----------+
|        114301|      01-06-2023|        11:33:29|       3|       Astoria|        45|              1|         3|         3|             Tea|Brewed herbal tea|    Peppermint|Large|      June| Thursday|  11|    6|          3|
|        115405|      02-06-2023|        11:18:24|       3|       Astoria|        45|              1|   

In [13]:
from pyspark.sql.functions import to_timestamp, col, date_trunc

coffee_df.where((col("transaction_date") <= "05-06-2023") & (col("transaction_date") >= "03-06-2023")).count()

10502

In [5]:
sc = spark.sparkContext

sc._jsc.hadoopConfiguration().set("fs.s3a.access.key", "dev_user_01")
sc._jsc.hadoopConfiguration().set("fs.s3a.secret.key", "devpassword01")
sc._jsc.hadoopConfiguration().set("fs.s3a.endpoint", "http://localhost:9000")
sc._jsc.hadoopConfiguration().set("fs.s3a.path.style.access", "true")
sc._jsc.hadoopConfiguration().set("fs.s3a.connection.ssl.enabled", "false")
sc._jsc.hadoopConfiguration().set("fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem")
sc._jsc.hadoopConfiguration().set("fs.s3a.connection.ssl.enabled", "false")

In [6]:
coffee_df \
    .write \
    .mode("overwrite") \
    .parquet(f"s3a://{bucket_name}/coffee_sales.parquet")

24/03/31 17:35:36 WARN MetricsConfig: Cannot locate configuration: tried hadoop-metrics2-s3a-file-system.properties,hadoop-metrics2.properties


In [14]:
coffee_df.where((col("transaction_date") <= "05-06-2023") & (col("transaction_date") >= "03-06-2023")) \
    .write \
    .partitionBy("store_location") \
    .mode("overwrite") \
    .parquet(f"s3a://{bucket_name}/coffee_partition.parquet")

In [4]:
bike_csv_dataset = 'mibici_2014-2024.csv'
bike_df = spark.read.format('csv').option('header', 'true').load(bike_csv_dataset)
#bike_df.count()

In [5]:
from pyspark.sql.functions import to_timestamp, col, date_trunc

bike_df = bike_df \
    .withColumn("Trip_start",to_timestamp(col("Trip_start"),"yyyy-MM-dd HH:mm:ss")) \
    .withColumn("Trip_end",to_timestamp(col("Trip_end"),"yyyy-MM-dd HH:mm:ss")) \
    .where(col("Trip_start") >= "2024-01-21").cache()

#bike_df.select(col("Trip_start"), date_trunc("day", col("Trip_start"))).show(2)
#bike_df.show(2)

In [8]:
bike_df.count()

ConnectionRefusedError: [Errno 111] Connection refused

In [7]:
bike_df.where(col("Trip_start") >= "2023-12-01").count()

683184

In [5]:
sc = spark.sparkContext

sc._jsc.hadoopConfiguration().set("fs.s3a.access.key", "dev_user_01")
sc._jsc.hadoopConfiguration().set("fs.s3a.secret.key", "devpassword01")
sc._jsc.hadoopConfiguration().set("fs.s3a.endpoint", "http://localhost:9000")
sc._jsc.hadoopConfiguration().set("fs.s3a.path.style.access", "true")
sc._jsc.hadoopConfiguration().set("fs.s3a.connection.ssl.enabled", "false")
sc._jsc.hadoopConfiguration().set("fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem")
sc._jsc.hadoopConfiguration().set("fs.s3a.connection.ssl.enabled", "false")

In [7]:
bike_df.select(col("Trip_Id"), col("User_Id"), col("Sex"), col("Birth_year"),
               date_trunc("day", col("Trip_start")).alias("start_date"), col("Trip_start"),
               col("Trip_end"), col("Origin_Id"), col("Duration"),
               col("Age"), col("Destination_Id")) \
    .write \
    .mode("overwrite") \
    .parquet(f"s3a://{bucket_name}/bike_dataset.parquet")

24/03/31 16:12:56 WARN MetricsConfig: Cannot locate configuration: tried hadoop-metrics2-s3a-file-system.properties,hadoop-metrics2.properties
24/03/31 16:13:02 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , Trip_Id, User_Id, Sex, Birth_year, Trip_start, Trip_end, Origin_Id, Destination_Id, Age, Duration
 Schema: _c0, Trip_Id, User_Id, Sex, Birth_year, Trip_start, Trip_end, Origin_Id, Destination_Id, Age, Duration
Expected: _c0 but found: 
CSV file: file:///home/ihor_lukianov/pyspark/mibici_2014-2024.csv


In [7]:
bike_df \
    .write \
    .partitionBy("Origin_Id") \
    .mode("overwrite") \
    .parquet(f"s3a://{bucket_name}/bike_partition.parquet")

24/03/31 17:19:21 WARN MetricsConfig: Cannot locate configuration: tried hadoop-metrics2-s3a-file-system.properties,hadoop-metrics2.properties
24/03/31 17:19:34 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , Trip_Id, User_Id, Sex, Birth_year, Trip_start, Trip_end, Origin_Id, Destination_Id, Age, Duration
 Schema: _c0, Trip_Id, User_Id, Sex, Birth_year, Trip_start, Trip_end, Origin_Id, Destination_Id, Age, Duration
Expected: _c0 but found: 
CSV file: file:///home/ihor_lukianov/pyspark/mibici_2014-2024.csv
ERROR:root:KeyboardInterrupt while sending command.                (0 + 1) / 19]
Traceback (most recent call last):
  File "/home/ihor_lukianov/pyspark/env/lib/python3.10/site-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/home/ihor_lukianov/pyspark/env/lib/python3.10/site-packages/py4j/clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
  Fi

KeyboardInterrupt: 

### Iceberg

In [1]:
from pyspark import SparkConf
from pyspark.sql import SparkSession

DEPENDENCIES = "org.apache.iceberg:iceberg-spark-runtime-3.3_2.12:1.3.1"
DEPENDENCIES += ",org.postgresql:postgresql:42.6.0"
DEPENDENCIES += ",software.amazon.awssdk:bundle:2.20.18"
DEPENDENCIES += ",com.amazonaws:aws-java-sdk-bundle:1.11.901"
DEPENDENCIES += ",org.apache.hadoop:hadoop-aws:3.3.4"

minio_access_key = "dev_user_01"
minio_secret_key = "devpassword01"
minio_end_point = "http://localhost:9000"

pg_user = "postgres"
pg_password = "postgres"
pg_db = "iceberg"

icb_catalog_name = 'pg_catalog'
minio_bucket = "iceberg"
iceberg_warehouse = 'iceberg-pg'
storage_type = 'archives'


conf = (
    SparkConf()
    .setAppName('iceberg_minio')
    #Dependencies
    .set('spark.jars.packages', DEPENDENCIES)
    #SQL Extensions
    .set('spark.sql.extensions', 'org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions')
    #Catalog Configuration
    .set(f'spark.sql.catalog.{icb_catalog_name}', 'org.apache.iceberg.spark.SparkCatalog')
    .set(f'spark.sql.catalog.{icb_catalog_name}.catalog-impl', 'org.apache.iceberg.jdbc.JdbcCatalog')
    .set(f'spark.sql.catalog.{icb_catalog_name}.uri', f'jdbc:postgresql://localhost:5439/{pg_db}')
    .set(f'spark.sql.catalog.{icb_catalog_name}.jdbc.user', pg_user)
    .set(f'spark.sql.catalog.{icb_catalog_name}.jdbc.password', pg_password)
    .set(f'spark.sql.catalog.{icb_catalog_name}.jdbc.verifyServerCertificate', 'true')
    .set(f'spark.sql.catalog.{icb_catalog_name}.jdbc.useSSL', 'true')
    .set(f'spark.sql.defaultCatalog', icb_catalog_name)
    .set(f'spark.sql.catalog.{icb_catalog_name}.warehouse', f's3a://{minio_bucket}/{iceberg_warehouse}/{storage_type}/')
    # MinIO Configuration
    .set('spark.hadoop.fs.s3a.access.key', minio_access_key)
    .set('spark.hadoop.fs.s3a.secret.key', minio_secret_key)
    .set("spark.hadoop.fs.s3a.endpoint", minio_end_point)
    .set("spark.hadoop.fs.s3a.path.style.access", "true")
    .set("spark.hadoop.fs.s3a.connection.ssl.enabled", "false")
    .set("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem")
    .set("spark.hadoop.fs.s3a.connection.ssl.enabled", "false")
    )

spark = SparkSession.builder.config(conf=conf).getOrCreate()

your 131072x1 screen size is bogus. expect trouble
24/03/31 21:19:11 WARN Utils: Your hostname, DESKTOP-IEOE9QF resolves to a loopback address: 127.0.1.1; using 172.23.154.177 instead (on interface eth0)
24/03/31 21:19:11 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


:: loading settings :: url = jar:file:/home/ihor_lukianov/pyspark/env/lib/python3.10/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/ihor_lukianov/.ivy2/cache
The jars for the packages stored in: /home/ihor_lukianov/.ivy2/jars
org.apache.iceberg#iceberg-spark-runtime-3.3_2.12 added as a dependency
org.postgresql#postgresql added as a dependency
software.amazon.awssdk#bundle added as a dependency
com.amazonaws#aws-java-sdk-bundle added as a dependency
org.apache.hadoop#hadoop-aws added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-40b41d22-0e22-4393-929b-89a5f3d499e5;1.0
	confs: [default]
	found org.apache.iceberg#iceberg-spark-runtime-3.3_2.12;1.3.1 in central
	found org.postgresql#postgresql;42.6.0 in central
	found org.checkerframework#checker-qual;3.31.0 in central
	found software.amazon.awssdk#bundle;2.20.18 in central
	found software.amazon.eventstream#eventstream;1.0.1 in central
	found com.amazonaws#aws-java-sdk-bundle;1.11.901 in central
	found org.apache.hadoop#hadoop-aws;3.3.4 in central
	found com.amazonaws#aws-java-sdk-bundle;1.12.262 in

In [2]:
spark.sql(f"""CREATE OR REPLACE TABLE {icb_catalog_name}.TEST_SCHEMA.TEST_TABLE_MINIO_PG (
             FIELD_1 BIGINT,
             FIELD_2 varchar(50),
             FIELD_3 DATE,
             FIELD_4 DOUBLE,
             FIELD_5 TIMESTAMP
             )
             USING iceberg
             """)

# Displays Tables Created In The PG Catalog (TEST_SCHEMA)
spark.sql(f'SHOW TABLES IN {icb_catalog_name}.TEST_SCHEMA').show(truncate=False)

Py4JJavaError: An error occurred while calling o61.sql.
: java.lang.NoClassDefFoundError: org/apache/spark/sql/catalyst/expressions/AnsiCast
	at org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions.$anonfun$apply$6(IcebergSparkSessionExtensions.scala:54)
	at org.apache.spark.sql.SparkSessionExtensions.$anonfun$buildResolutionRules$1(SparkSessionExtensions.scala:175)
	at scala.collection.TraversableLike.$anonfun$map$1(TraversableLike.scala:286)
	at scala.collection.mutable.ResizableArray.foreach(ResizableArray.scala:62)
	at scala.collection.mutable.ResizableArray.foreach$(ResizableArray.scala:55)
	at scala.collection.mutable.ArrayBuffer.foreach(ArrayBuffer.scala:49)
	at scala.collection.TraversableLike.map(TraversableLike.scala:286)
	at scala.collection.TraversableLike.map$(TraversableLike.scala:279)
	at scala.collection.AbstractTraversable.map(Traversable.scala:108)
	at org.apache.spark.sql.SparkSessionExtensions.buildResolutionRules(SparkSessionExtensions.scala:175)
	at o

In [1]:
from pyspark import SparkConf
from pyspark.sql import SparkSession

minio_access_key = "dev_user_01"
minio_secret_key = "devpassword01"
minio_end_point = "http://localhost:9000"

conf = (
    SparkConf()
    .setAppName('delta_minio')
    #Dependencies
    .set('spark.jars.packages', "io.delta:delta-spark_2.12:3.1.0,org.apache.hadoop:hadoop-aws:3.3.4")
    #SQL Extensions
    .set('spark.sql.extensions', 'io.delta.sql.DeltaSparkSessionExtension')
    #Catalog Configuration
    # .set(f'spark.sql.catalog.{icb_catalog_name}', 'org.apache.iceberg.spark.SparkCatalog')
    # .set(f'spark.sql.catalog.{icb_catalog_name}.catalog-impl', 'org.apache.iceberg.jdbc.JdbcCatalog')
    # .set(f'spark.sql.catalog.{icb_catalog_name}.uri', f'jdbc:postgresql://localhost:5439/{pg_db}')
    # .set(f'spark.sql.catalog.{icb_catalog_name}.jdbc.user', pg_user)
    # .set(f'spark.sql.catalog.{icb_catalog_name}.jdbc.password', pg_password)
    # .set(f'spark.sql.catalog.{icb_catalog_name}.jdbc.verifyServerCertificate', 'true')
    # .set(f'spark.sql.catalog.{icb_catalog_name}.jdbc.useSSL', 'true')
    # .set(f'spark.sql.defaultCatalog', icb_catalog_name)
    # .set(f'spark.sql.catalog.{icb_catalog_name}.warehouse', f's3a://{minio_bucket}/{iceberg_warehouse}/{storage_type}/')
    # MinIO Configuration
    .set('spark.hadoop.fs.s3a.access.key', minio_access_key)
    .set('spark.hadoop.fs.s3a.secret.key', minio_secret_key)
    .set("spark.hadoop.fs.s3a.endpoint", minio_end_point)
    .set("spark.databricks.delta.retentionDurationCheck.enabled", "false")
    .set("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")
    .set("spark.hadoop.fs.s3a.path.style.access", "true")
    .set("spark.hadoop.fs.s3a.connection.ssl.enabled", "false")
    .set("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem")
    .set("spark.hadoop.fs.s3a.connection.ssl.enabled", "false")
    )

spark = SparkSession.builder.config(conf=conf).getOrCreate()

your 131072x1 screen size is bogus. expect trouble
24/03/31 21:41:56 WARN Utils: Your hostname, DESKTOP-IEOE9QF resolves to a loopback address: 127.0.1.1; using 172.23.154.177 instead (on interface eth0)
24/03/31 21:41:56 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


:: loading settings :: url = jar:file:/home/ihor_lukianov/pyspark/env/lib/python3.10/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/ihor_lukianov/.ivy2/cache
The jars for the packages stored in: /home/ihor_lukianov/.ivy2/jars
io.delta#delta-spark_2.12 added as a dependency
org.apache.hadoop#hadoop-aws added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-3a0edf8c-0a09-4966-954e-3234d1c8a3fd;1.0
	confs: [default]
	found io.delta#delta-spark_2.12;3.1.0 in central
	found io.delta#delta-storage;3.1.0 in central
	found org.antlr#antlr4-runtime;4.9.3 in central
	found org.apache.hadoop#hadoop-aws;3.3.4 in central
	found com.amazonaws#aws-java-sdk-bundle;1.12.262 in central
	found org.wildfly.openssl#wildfly-openssl;1.0.7.Final in central
:: resolution report :: resolve 724ms :: artifacts dl 35ms
	:: modules in use:
	com.amazonaws#aws-java-sdk-bundle;1.12.262 from central in [default]
	io.delta#delta-spark_2.12;3.1.0 from central in [default]
	io.delta#delta-storage;3.1.0 from central in [default]
	org.antlr#antlr4-runtime;4.9.3 from central in [default]
	o

In [2]:
from pyspark.sql.functions import to_timestamp, col, date_trunc

minio_bucket = "delta"

coffee_csv = 'coffee_sales.csv'
coffee_df = spark.read.format('csv').option('header', 'true').load(coffee_csv)
coffee_df.count()

Py4JJavaError: An error occurred while calling o50.load.
: java.lang.NoClassDefFoundError: org/apache/spark/sql/catalyst/analysis/UnresolvedLeafNode
	at java.base/java.lang.ClassLoader.defineClass1(Native Method)
	at java.base/java.lang.ClassLoader.defineClass(ClassLoader.java:1017)
	at java.base/java.security.SecureClassLoader.defineClass(SecureClassLoader.java:174)
	at java.base/jdk.internal.loader.BuiltinClassLoader.defineClass(BuiltinClassLoader.java:800)
	at java.base/jdk.internal.loader.BuiltinClassLoader.findClassOnClassPathOrNull(BuiltinClassLoader.java:698)
	at java.base/jdk.internal.loader.BuiltinClassLoader.loadClassOrNull(BuiltinClassLoader.java:621)
	at java.base/jdk.internal.loader.BuiltinClassLoader.loadClass(BuiltinClassLoader.java:579)
	at java.base/jdk.internal.loader.ClassLoaders$AppClassLoader.loadClass(ClassLoaders.java:178)
	at java.base/java.lang.ClassLoader.loadClass(ClassLoader.java:522)
	at java.base/java.lang.ClassLoader.defineClass1(Native Method)
	at java.b

In [3]:
coffee_df.where((col("transaction_date") <= "05-06-2023") & (col("transaction_date") >= "03-06-2023")) \
    .write \
    .format("delta") \
    .save(f"s3a://{minio_bucket}/coffee-delta")

24/03/31 21:34:00 WARN MetricsConfig: Cannot locate configuration: tried hadoop-metrics2-s3a-file-system.properties,hadoop-metrics2.properties
ERROR:root:KeyboardInterrupt while sending command.
Traceback (most recent call last):
  File "/home/ihor_lukianov/pyspark/env/lib/python3.10/site-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/home/ihor_lukianov/pyspark/env/lib/python3.10/site-packages/py4j/clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
  File "/usr/lib/python3.10/socket.py", line 705, in readinto
    return self._sock.recv_into(b)
KeyboardInterrupt


KeyboardInterrupt: 